In [1]:
import requests
import env
import os
import pandas as pd
import json
from env import Client_ID
from igdb.wrapper import IGDBWrapper


#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Connecting to API

In [ ]:
url = env.get_db_url()
response = requests.post(url)
print(response.text)


In [ ]:
data = response.json()
data.keys()

In [ ]:
access_token = data['access_token']

In [ ]:
# verify access token matches
access_token == data['access_token']

In [ ]:
requests.post(url)

In [ ]:
#function to connect to api and get access token
#requests.post(url) != 200
url = env.get_db_url()
response = requests.post(url)
data = response.json()
access_token = data['access_token']


In [ ]:
def connect_api():
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    return access_token

### Making requests from API

- Most of the requests to the API will use the POST method
- The base URL is: https://api.igdb.com/v4
- You define which endpoint you wish to query by appending /{endpoint name} to the base URL eg. https://api.igdb.com/v4/games
- Include Client ID and access_token in the HEADER of your request so that your headers look like the following. Take special care of the capitalisation. Bearer should be hard-coded.

    ''' 
    Client-ID: Client ID

    Authorization: Bearer access_token 
    '''

- You use the BODY of your request to specify the fields you want to retrieve as well as any other filters, sorting etc

**Example**

    '''
    POST https://api.igdb.com/v4/games
    Client-ID: abcdefg12345
    Authorization: Bearer access12345token
    fields *;
    '''

more examples https://api-docs.igdb.com/#examples 

In [ ]:
url = env.get_db_url()
base_url = 'https://api.igdb.com/v4'
response = requests.post(base_url)
print(response.text)

In [ ]:
url = env.get_db_url()
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
base_url = 'https://api.igdb.com/v4'
response = requests.post(base_url , headers=headers)
print(response.text)

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; popularity;sort popularity desc;'
r = requests.post('https://api.igdb.com/v4/games' ,data=data ,  headers=headers)

In [ ]:
r.json()

In [ ]:
response.json()

In [ ]:
r.text

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; rating; sort rating asc;'
r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )

In [ ]:
wrapper = IGDBWrapper(f'{Client_ID}', f'{access_token}')

In [ ]:
'''With a wrapper instance already created'''
# JSON API request
byte_array = wrapper.api_request(
            'games',
            'fields *; limit 500;'
          )
# parse into JSON however you like...

# Protobuf API request
from igdb.igdbapi_pb2 import GameResult
byte_array = wrapper.api_request(
            'games.pb', # Note the '.pb' suffix at the endpoint
            'fields *; limit 500;'
          )
games_message = GameResult()
games_message.ParseFromString(byte_array) # Fills the protobuf message object with the response

In [ ]:
byte_array = wrapper.api_request(
            'games',
            'fields *; limit 500;'
          )

In [ ]:
testgames = []
for i in range(0, 500):
    games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 500};')
    testgames.append(games)

In [ ]:
testgames[0]

In [ ]:
pd.DataFrame(testgames.json())

In [ ]:
byte_array

In [ ]:
y = json.loads(byte_array)

In [ ]:
pd.DataFrame(y).head()

In [ ]:
pd.DataFrame(y).columns

In [ ]:
pd.DataFrame(y['id', 'category', 'genres', 'name', 'parent_game', 'slug','summary', 'tags', 'themes', 'url', 'websites',])

In [ ]:

pd.DataFrame(y).head(1)

In [ ]:
pd.DataFrame(y).url

In [ ]:
y

In [ ]:
games_message.ParseFromString(byte_array)

In [ ]:
type(games_message)

In [ ]:
games_message[1]

get ratings

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
#data = 'fields *; popularity;sort popularity desc;'
requests.get('https://api.igdb.com/v4/age_ratings' ,  headers=headers).text

### create acquire games library function

In [2]:
def connect_api():
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    return access_token

In [4]:
access_token = connect_api()

In [5]:
def connect_to_games():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; popularity;sort popularity desc;'
    r = requests.post('https://api.igdb.com/v4/games' ,data=data ,  headers=headers)
    return r

In [11]:
tf = connect_to_games()

In [6]:
def run_wrapper():
    wrapper = IGDBWrapper(f'{Client_ID}', f'{access_token}')
    return wrapper

In [8]:
wrapper = run_wrapper()

In [9]:
def get_game_library(wrapper):
    testgames2 = []
    for i in range(0, 10):
        games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 10};')
        testgames2.append(games)
    return testgames2

In [10]:
tf = get_game_library(wrapper)

In [12]:
# function that puts response list object into a dataframe for each page
def game_library2(wrapper):
    game_library = pd.DataFrame()
    for i in range (0, 409):
        games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(games)
        results_df =pd.DataFrame(y)
        game_library = pd.concat([game_library, results_df])
    return game_library

In [13]:
game_library2 = game_library2(wrapper)

In [ ]:
tf.json()

In [ ]:
tf[0]

In [ ]:
y = json.loads(tf[0])

In [ ]:
y

def _game_library(wrapper):
    testgames2 = []
    for i in range(0, 10):
        games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 10};')
        testgames2.append(games)
        json.loads(tf[0])
    return testgames2

In [74]:
game_library2.head()

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,dlcs,ports,expansions,remakes,expanded_games,remasters,forks,standalone_expansions
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...","[13196, 18623, 28244, 32510, 36258, 55036, 791...",panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,"[22, 268435471, 268435472]",[22],1654096076,https://www.igdb.com/games/panzer-corps-2-axis...,[68880],"[285491, 286755]",2992a43b-4d59-7ab8-e500-71a352404759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,0,NaN,1498435200,"[20127, 1988884]",[5],Dogou Souken,NaN,NaN,"[12364, 27270, 43367, 103281, 103292, 103298, ...",dogou-souken,An overhead-view shoot'em up game.,[268435461],NaN,1604620800,https://www.igdb.com/games/dogou-souken,NaN,NaN,5e7446ff-58d9-e744-f0d3-5e10241304c1,5.364576e+08,[52],[91007],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",[9],City Mysteries,NaN,"[155025, 155026, 155027]","[10603, 19222, 25905, 41349, 86974, 87507, 885...",city-mysteries,"Search buildings, streets, landmarks as you se...",[268435465],NaN,1641425677,https://www.igdb.com/games/city-mysteries,NaN,NaN,87b342e7-b912-ca8d-4ffb-f91f9e4f1870,1.404346e+09,[37],[138333],[37944],"[130531, 130532]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]","[31, 32]",Sword of the Black Stone,NaN,"[231528, 231529, 231530, 231531, 231532]","[25646, 29783, 68271, 80916, 96217, 105269, 10...",sword-of-the-black-stone,If your looking for action and adventure then ...,"[1, 268435487, 268435488]",[1],1643857546,https://www.igdb.com/games/sword-of-the-black-...,NaN,[84617],0b575316-96dc-b056-86e0-c958c1c95efb,1.527120e+09,[6],[188385],NaN,"[155850, 155852]",[1],[2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,[33736],0,169077.0,1621959514,"[2052375, 2057300]","[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,"[463145, 463148, 463149, 463151, 463152]","[25311, 26145, 28277, 33603, 51577, 65827, 791...",bok-bok-a-chicken-dating-sim,Have you ever pondered what the romantic life ...,"[268435469, 268435471, 268435487, 268435488]",NaN,1643850601,https://www.igdb.com/games/bok-bok-a-chicken-d...,NaN,[192858],9585e273-2606-970f-3088-60261ea77068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[63465],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
game_library2.to_csv("games.csv")

### get genres

In [15]:
access_token = connect_api()

In [16]:
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/genres' ,data=data ,  headers=headers)

In [17]:
r

<Response [200]>

In [26]:
genres = wrapper.api_request('genres', 'fields *;limit 500;')

In [27]:
x = json.loads(genres)

In [28]:
genres =pd.DataFrame(x)

In [29]:
genres

,id,created_at,name,slug,updated_at,url,checksum
0,4,1297555200,Fighting,fighting,1323216000,https://www.igdb.com/genres/fighting,2ccc6572-bdde-6ed4-8843-25447ea40782
1,5,1297555200,Shooter,shooter,1323216000,https://www.igdb.com/genres/shooter,bb15fd3f-0f46-e5f3-2b40-d046cf9bd2ef
2,7,1297555200,Music,music,1323216000,https://www.igdb.com/genres/music,22d44a0d-89c7-580f-eef2-e06f178fdd47
3,8,1297555200,Platform,platform,1323216000,https://www.igdb.com/genres/platform,697fc5a4-b96f-a803-288a-498bd5dd1de1
4,9,1297555200,Puzzle,puzzle,1323216000,https://www.igdb.com/genres/puzzle,616de9c3-8a00-0232-9df9-00014cfac51b
5,10,1297555200,Racing,racing,1323216000,https://www.igdb.com/genres/racing,73c2653f-54d8-c1bd-d523-dd57fb753194
6,11,1297555200,Real Time Strategy (RTS),real-time-strategy-rts,1323216000,https://www.igdb.com/genres/real-time-strategy...,aaa36cbc-2258-8653-a461-1358df8ce445
7,12,1297555200,Role-playing (RPG),role-playing-rpg,1323216000,https://www.igdb.com/genres/role-playing-rpg,42dea3b2-7fe2-e734-91cd-f80ce62a14c3
8,13,1297555200,Simulator,simulator,1323216000,https://www.igdb.com/genres/simulator,9779772a-f08f-9e8e-bd26-70c9eecc34e8
9,14,1297555200,Sport,sport,1323216000,https://www.igdb.com/genres/sport,e598c458-3e21-7a14-50a5-ea53733ab22f


### get age ratings

In [30]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/age_ratings' ,data=data ,  headers=headers)

age_ratings = wrapper.api_request('age_ratings', 'fields *;limit 500;')

x = json.loads(age_ratings)

age_ratings =pd.DataFrame(x)

In [33]:
def get_age_ratings(wrapper):
    age_ratings = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('age_ratings', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        age_ratings = pd.concat([age_ratings, results_df])
    return age_ratings

In [35]:
age_ratings = get_age_ratings(wrapper)
age_ratings

,id,category,rating,checksum,content_descriptions,synopsis
0,22474,1,8,361fd414-94d8-d6f8-6641-47abd3e8a75c,NaN,NaN
1,43430,1,8,5970434e-a37f-d43d-8975-2b23ab3370b3,NaN,NaN
2,2853,1,10,2d21809b-c282-95ba-69fb-5eda4d684b43,"[4244, 4245, 4246, 4247]",NaN
3,32550,1,9,40557417-e2a9-9a4f-5810-4159463467a1,[32972],NaN
4,65037,3,15,9c0f7e9a-8ceb-27f3-3514-cc6407a176cd,"[66283, 66284, 66285]",NaN
...,...,...,...,...,...,...
452,91721,5,23,a2168458-26e5-d2bd-e41c-92da413796a6,[95693],NaN
453,91707,5,26,e69ec952-2a88-2a0b-a988-c0b77861e5a8,NaN,NaN
454,91703,5,23,0276240f-5aec-d987-f1fb-a758930a0cd6,NaN,NaN
455,91705,5,25,46caae34-d5b4-e33e-84e8-a9b3f584f66d,[95677],NaN


In [72]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields content_descriptions; '
r = requests.post('https://api.igdb.com/v4/age_ratings' ,data=data ,  headers=headers)

In [73]:
r.json()

[{'id': 22474},
 {'id': 43430},
 {'id': 2853, 'content_descriptions': [4244, 4245, 4246, 4247]},
 {'id': 32550, 'content_descriptions': [32972]},
 {'id': 65037, 'content_descriptions': [66283, 66284, 66285]},
 {'id': 10, 'content_descriptions': [24, 25]},
 {'id': 65004, 'content_descriptions': [66215, 66216, 66217]},
 {'id': 13683},
 {'id': 78460},
 {'id': 70534, 'content_descriptions': [72442, 72443, 72444, 72445, 72446]}]

In [65]:
 y = json.loads(r)

TypeError: the JSON object must be str, bytes or bytearray, not Response

In [75]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; '
r = requests.post('https://api.igdb.com/v4/age_ratings/age_ratings/age_rating_enumns' ,data=data ,  headers=headers)

age_ratings = wrapper.api_request('age_ratings' , 'age_ratings_enumns', 'fields *;limit 500;')

x = json.loads(age_ratings)

age_ratings =pd.DataFrame(x)

TypeError: api_request() takes 3 positional arguments but 4 were given

In [ ]:
endpoints = [
    'https://api.igdb.com/v4/age_ratings',
    'https://api.igdb.com/v4/age_rating_content_descriptions',
    'https://api.igdb.com/v4/characters',
    'https://api.igdb.com/v4/companies',
    'https://api.igdb.com/v4/collections',
    'https://api.igdb.com/v4/franchises',
    'https://api.igdb.com/v4/game_engines'
    ]

### Get characters

In [76]:
access_token = connect_api()

In [80]:
def get_characters(wrapper):
    characters = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('characters', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        characters = pd.concat([characters, results_df])
    return characters

In [81]:
characters = get_characters(wrapper)
characters

,id,created_at,games,name,slug,updated_at,url,checksum,gender,mug_shot,species,description,akas
0,13116,1555977600,[103020],Nuclear Launch Silo,nuclear-launch-silo,1555977600,https://www.igdb.com/characters/nuclear-launch...,bb6801a9-2b8f-6843-1964-2f440f1b8e48,NaN,NaN,NaN,NaN,NaN
1,14447,1562198400,[75242],Agent Carson,agent-carson,1562198400,https://www.igdb.com/characters/agent-carson,ac16307c-58d1-37cd-cda7-490f9345bb7b,NaN,NaN,NaN,NaN,NaN
2,504,1397865600,"[2765, 2911]",Gary,gary,1501632000,https://www.igdb.com/characters/gary,56dfd9a7-5cfa-ee91-14e5-46a493843313,NaN,NaN,NaN,NaN,NaN
3,2692,1414800000,[41],O,o,1458518400,https://www.igdb.com/characters/o,e32722d2-9798-5a72-b240-fdcbe43fabc5,NaN,NaN,NaN,NaN,NaN
4,6715,1449878400,[1958],Thophile Paddington,thophile-paddington,1449878400,https://www.igdb.com/characters/thophile-paddi...,51e57c03-5b7e-fc22-c838-1f7dfe753e92,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,16575,1647897710,[2369],Menelag,menelag,1647897729,https://www.igdb.com/characters/menelag,ecc219c8-c15f-7ee2-4bb5-8287ff53b418,NaN,NaN,NaN,NaN,NaN
392,16579,1647897711,[2369],Additional,additional,1647897732,https://www.igdb.com/characters/additional,075e65d8-c756-a0c9-9224-086ccfd9cfe3,NaN,NaN,NaN,NaN,NaN
393,4629,1432043872,[897],Cate Archer,cate-archer,1651237111,https://www.igdb.com/characters/cate-archer,1dbdc5b9-346d-5870-20c7-3627a132292e,1.0,4068.0,1.0,NaN,"[Catherine Ann Archer, The Fox]"
394,10236,1475914520,NaN,Funtime Freddy,funtime-freddy,1657731347,https://www.igdb.com/characters/funtime-freddy,3b31ca09-177a-0c37-eb02-06f979aa5f96,0.0,4069.0,4.0,Funtime Freddy is a major antagonist who first...,NaN
